In [62]:
# Final Code
import pandas as pd
import numpy as np
from gurobipy import Model, GRB

def optimizeShipment(inputFile, outputFile):
    fulfillment_centers = pd.read_excel(inputFile, sheet_name = "Fulfilment Centers",index_col = 0)
    regions = pd.read_excel(inputFile, sheet_name = "Regions", index_col = 0)
    delta = pd.read_excel(inputFile, sheet_name = "Distances", index_col = 0)
    items = pd.read_excel(inputFile, sheet_name = "Items", index_col = 0)
    d = pd.read_excel(inputFile, sheet_name = "Demand", index_col = 0)
    I = fulfillment_centers.index
    J = regions.index
    K = items.index
    w = list(items['shipping_weight'])
    s = list(items['storage_size'])
    q = pd.Series(index=fulfillment_centers.index,data=fulfillment_centers.capacity)
    mod = Model()
    x = mod.addVars(I,J,K,vtype = GRB.CONTINUOUS,name = 'x')
    mod.setObjective(1.38*sum(x[i,j,k]*delta.loc[j,i]*w[k] for i in I for j in J for k in K), sense = GRB.MINIMIZE)    # shift requirement
    # Capacity Constraint
    for i in I:
        mod.addConstr(sum(s[k]*x[i,j,k] for j in J for k in K) <= q[i])
    # Demand Constrain
    for j in J:
        for k in K:
            mod.addConstr(sum(x[i,j,k] for i in I) >= d.loc[k,j])
    mod.setParam("OutputFlag", False)
    mod.setParam('MIPGap',1e-6)
    mod.optimize()
    print(f'Finished Optimizing! Objective Value: {mod.objVal}')
    # Saving to excel
    df1 = pd.DataFrame({'Minimum Total Cost':[mod.objVal]})
    df2 = pd.DataFrame({'FC_Name':[],'region_ID':[],'item_ID':[],'Shipment':[]})
    fc_name = []
    region_id = []
    item_id = []
    shipment = []
    for i in I:
        for j in J:
            for k in K:
                if x[i,j,k].x > 0:
                    fc_name.append(i)
                    region_id.append(j)
                    item_id.append(k)
                    shipment.append(x[i,j,k].x)
    df2 = pd.DataFrame({'FC_Name':fc_name,'region_ID':region_id,'item_ID':item_id,'Shipment':shipment})
    writer = pd.ExcelWriter(outputFile)
    df1.to_excel(writer, sheet_name = "Summary", index = False)
    df2.to_excel(writer, sheet_name = "Solution", index = False)
    writer.save()

In [63]:
optimizeShipment('12-retail-toy-input.xlsx','12-retail-toy-myOutput.xlsx')

Finished Optimizing! Objective Value: 3400.769189999999


In [64]:
optimizeShipment('12-retail-real-input.xlsx','12-retail-real-myOutput.xlsx')

Finished Optimizing! Objective Value: 9841229.288170151
